# **Code Generation and AutoDiff with a Cart-Pole System in Crocoddyl**

In this notebook, we demonstrate how to perform code generation and compute automatic differentiation (AutoDiff) derivatives for an action model written in Python. Specifically, we develop an action model for a cart-pole system, similar to the one in the `cartpole_swingup.ipynb` notebook. The key features of this notebook are
 - **Understanding Code Generation**: Implement the cartpole dynamics and its analytical derivates using Crocoddyl's code generation framework.
 - **Automatic Differentiation**: Validate analytical derivatives with AutoDiff and compare computational performance.
 - **Optimal Control with Code-Generated Actions**: Solves an optimal control problem using the `SolveFDDP` solver and analyze the computational speedup from code generation.

## Overview of the Cart-Pole System

The cart-pole system consists of a cart that can move along a horizontal track and a pole attached to the cart via a pivot joint. The goal is to balance the pole in the upright position by applying forces to the cart.

<div style="text-align:center;">
    <img src="https://www.ashwinnarayan.com/img/cart-pole.png" width="300" height="300">
</div>

## Equations of Motion

The state of the system is given by:

- $y$: Position of the cart
- $\theta$: Angle of the pole from the upright position
- $\dot{y}$: Velocity of the cart
- $\dot{\theta}$: Angular velocity of the pole

For more details on the equation of motions, integrator, and cost functions please go to the `cartpole_swingup.ipynb` notebook.

## Code Generation in Crocoddyl

When enabling `BUILD_WITH_CODEGEN_SUPPORT`, Crocoddyl leverages `CppADCodeGen` for efficient code generation and automatic differentiation of action models. To enable this, we modify the `ActionModelCartpole` from `cartpole_swingup.ipynb` as follows:
 1. Inheritance: Extend Crocoddyl’s base action model supporting `CppAD<CppAD::cg::CG<>>` types via the `cgfloat64` module.
 2. Function Replacement: Replace unsupported primitive functions with `CppADCodeGen` equivalents (e.g., `numpy.sin` → `pycppad.sin_me`).

The following section presents the modified cart-pole action model in detail.

In [ ]:
import crocoddyl.cgfloat64
from pycppad import ADCG, CG
import numpy as np

class ActionModelCartpole(crocoddyl.cgfloat64.ActionModelAbstract):
    """
    Defines the action model for the cart-pole system in Crocoddyl.
    
    This model simulates a cart moving along a horizontal track with a 
    freely rotating pole attached to it. The goal is to control the force 
    applied to the cart to stabilize the pole.
    
    State vector: [y, θ, ẏ, θ̇]
    Control input: [f] (force applied to the cart)
    """
    def __init__(self):
        """
        Initializes the cart-pole action model with system parameters and cost weights.
        """
        crocoddyl.cgfloat64.ActionModelAbstract.__init__(
            self, crocoddyl.cgfloat64.StateVector(4), nu=1, nr=6)

        self.Δt = ADCG(CG(5e-2))
        self.m_cart = ADCG(CG(1.0))
        self.m_pole = ADCG(CG(0.1))
        self.l_pole = ADCG(CG(0.5))
        self.grav = ADCG(CG(9.81))
        self.costWeights = [
            ADCG(CG(1.0)),
            ADCG(CG(1.0)),
            ADCG(CG(0.1)),
            ADCG(CG(0.001)),
            ADCG(CG(0.001)),
            ADCG(CG(1.0)),
        ]  # sin, 1-cos, x, xdot, thdot, f

    def calc(self, data, x, u=None):
        """
        Computes the next state and the cost function.

        Args:
            data (ActionDataCartpole): Data structure to store intermediate results.
            x (numpy.ndarray): State vector [y, θ, ẏ, θ̇].
            u (numpy.ndarray, optional): Control input [f]. Defaults to None.

        Updates:
            - Next state xnext.
            - Cost residuals r.
            - Total cost value.
        """
        if u is not None:
            # Getting the state and control variables
            y, θ, ẏ, θ̇, f = x[0], x[1], x[2], x[3], u[0]
            # Shortname for system parameters
            Δt, m_cart, m_pole, l_pole, grav, w = self.Δt, self.m_cart, self.m_pole, self.l_pole, self.grav, self.costWeights
            sin_θ, cos_θ = ADCG.sin_me(θ), ADCG.cos_me(θ)
            # Computing the cartpole dynamics
            data.μ = m_cart + m_pole * sin_θ* sin_θ
            data.ÿ̈ = f / data.μ
            data.ÿ̈ += m_pole * grav * cos_θ * sin_θ / data.μ
            data.ÿ̈ -= m_pole * l_pole * (sin_θ * θ̇ * θ̇ / data.μ)
            data.θ̈ = (f / l_pole) * cos_θ / data.μ
            data.θ̈ += ((m_cart + m_pole) * grav / l_pole) * sin_θ / data.μ
            data.θ̈ -= m_pole * cos_θ * sin_θ * θ̇ * θ̇ / data.μ
            data.ẏ_next = ẏ + Δt * data.ÿ̈
            data.θ̇_next = θ̇ + Δt * data.θ̈
            data.y_next = y + Δt * data.ẏ_next
            data.θ_next = θ + Δt * data.θ̇_next
            data.xnext[:] = np.array([data.y_next, data.θ_next, data.ẏ_next, data.θ̇_next])
            # Computing the cost residual and value
            data.r[:] = w * np.array([sin_θ, ADCG(CG(1.0)) - cos_θ, y, ẏ, θ̇, f])
            data.cost = ADCG(CG(0.0))
            for i in range(self.nr):
                data.cost += ADCG(CG(0.5)) * (data.r[i] * data.r[i])
        else:
            # Getting the state and control variables
            y, θ, ẏ, θ̇ = x[0], x[1], x[2], x[3]
            w = self.costWeights
            sin_θ, cos_θ = ADCG.sin_me(θ), ADCG.cos_me(θ)
            data.xnext[:] = x
            # Computing the cost residual and value
            data.r[:] = w * np.array([sin_θ, ADCG(CG(1.0)) - cos_θ, y, ẏ, θ̇, ADCG(CG(0.0))])
            data.cost = ADCG(CG(0.0))
            for i in range(self.nr):
                data.cost += ADCG(CG(0.5)) * (data.r[i] * data.r[i])

    def calcDiff(self, data, x, u=None):
        """
        Computes the derivatives of the dynamics and the cost function.

        Args:
            data (ActionDataCartpole): Data structure to store intermediate results.
            x (numpy.ndarray): State vector [y, θ, ẏ, θ̇].
            u (numpy.ndarray, optional): Control input [f]. Defaults to None.

        Updates:
            - Derivetives of the dynamics.
            - Derivatives of the cost function.
        """
        if u is not None:
            # Getting the state and control variables
            y, θ, ẏ, θ̇, f = x[0], x[1], x[2], x[3], u[0]
            # Shortname for system parameters
            Δt, m_cart, m_pole, l_pole, grav, w = self.Δt, self.m_cart, self.m_pole, self.l_pole, self.grav, self.costWeights
            sin_θ, cos_θ = ADCG.sin_me(θ), ADCG.cos_me(θ)
            # Computing the derivative of the cartpole dynamics
            data.dμ_dθ = ADCG(CG(2.0)) * m_pole * sin_θ * cos_θ
            data.dÿ̈_dy = ADCG(CG(0.0))
            data.dÿ̈_dθ = m_pole * grav * (cos_θ * cos_θ - sin_θ * sin_θ) / data.μ
            data.dÿ̈_dθ -= m_pole * l_pole * cos_θ * θ̇ * θ̇ / data.μ
            data.dÿ̈_dθ -= data.dμ_dθ * data.ÿ̈ / data.μ
            data.dÿ̈_dẏ = ADCG(CG(0.0))
            data.dÿ̈_dθ̇ = -ADCG(CG(2.0)) * m_pole * l_pole * sin_θ * θ̇ / data.μ
            data.dÿ̈_du = ADCG(CG(1.0)) / data.μ
            data.dθ̈_dy = ADCG(CG(0.0))
            data.dθ̈_dθ = -(f / l_pole) * sin_θ / data.μ
            data.dθ̈_dθ += ((m_cart + m_pole) * grav / l_pole) * cos_θ / data.μ
            data.dθ̈_dθ -= m_pole * (cos_θ * cos_θ - sin_θ * sin_θ) * θ̇ * θ̇ / data.μ
            data.dθ̈_dθ -= data.dμ_dθ * data.θ̈ / data.μ
            data.dθ̈_dẏ = ADCG(CG(0.0))
            data.dθ̈_dθ̇ = -ADCG(CG(2.0)) * m_pole * cos_θ * sin_θ * θ̇ / data.μ
            data.dθ̈_du = cos_θ / (l_pole * data.μ)
            data.dẏ_next_dy = Δt * data.dÿ̈_dy
            data.dẏ_next_dθ = Δt * data.dÿ̈_dθ
            data.dẏ_next_dẏ = ADCG(CG(1.0)) + Δt * data.dÿ̈_dẏ
            data.dẏ_next_dθ̇ = Δt * data.dÿ̈_dθ̇
            data.dẏ_next_du = Δt * data.dÿ̈_du
            data.dθ̇_next_dy = Δt * data.dθ̈_dy
            data.dθ̇_next_dθ = Δt * data.dθ̈_dθ
            data.dθ̇_next_dẏ = Δt * data.dθ̈_dẏ
            data.dθ̇_next_dθ̇ = ADCG(CG(1.0)) + Δt * data.dθ̈_dθ̇
            data.dθ̇_next_du = Δt * data.dθ̈_du
            data.dy_next_dy = ADCG(CG(1.0)) + Δt * data.dẏ_next_dy
            data.dy_next_dθ = Δt * data.dẏ_next_dθ
            data.dy_next_dẏ = Δt * data.dẏ_next_dẏ
            data.dy_next_dθ̇ = Δt * data.dẏ_next_dθ̇
            data.dy_next_du = Δt * data.dẏ_next_du
            data.dθ_next_dy = Δt * data.dθ̇_next_dy
            data.dθ_next_dθ = ADCG(CG(1.0)) + Δt * data.dθ̇_next_dθ
            data.dθ_next_dẏ = Δt * data.dθ̇_next_dẏ
            data.dθ_next_dθ̇ = Δt * data.dθ̇_next_dθ̇
            data.dθ_next_du = Δt * data.dθ̇_next_du
            data.Fx[:, :] = np.array([[data.dy_next_dy, data.dy_next_dθ, data.dy_next_dẏ, data.dy_next_dθ̇],
                                      [data.dθ_next_dy, data.dθ_next_dθ, data.dθ_next_dẏ, data.dθ_next_dθ̇],
                                      [data.dẏ_next_dy, data.dẏ_next_dθ, data.dẏ_next_dẏ, data.dẏ_next_dθ̇],
                                      [data.dθ̇_next_dy, data.dθ̇_next_dθ, data.dθ̇_next_dẏ, data.dθ̇_next_dθ̇]])
            data.Fu[:] = np.array([data.dy_next_du, data.dθ_next_du, data.dẏ_next_du, data.dθ̇_next_du])
            # Computing derivatives of the cost function
            w0_2, w1_2, w2_2, w3_2, w4_2, w5_2 = w[0] * w[0], w[1] * w[1], w[2] * w[2], w[3] * w[3], w[4] * w[4], w[5] * w[5]
            data.Lx[0] = w2_2 * y
            data.Lx[1] = w0_2 * sin_θ * cos_θ + w1_2 * (ADCG(CG(1.0)) - cos_θ) * sin_θ
            data.Lx[2] = w3_2 * ẏ
            data.Lx[3] = w4_2 * θ̇
            data.Lu[0] = w5_2 * f
            data.Lxx[0, 0] = w2_2
            data.Lxx[1, 1] = w0_2 * (cos_θ * cos_θ - sin_θ * sin_θ)
            data.Lxx[1, 1] += w1_2 * ((ADCG(CG(1.0)) - cos_θ) * cos_θ + sin_θ * sin_θ)
            data.Lxx[2, 2] = w3_2
            data.Lxx[3, 3] = w4_2
            data.Luu[:] = w5_2
        else:
            # Getting the state and control variables
            y, θ, ẏ, θ̇ = x[0], x[1], x[2], x[3]
            w = self.costWeights
            sin_θ, cos_θ = ADCG.sin_me(θ), ADCG.cos_me(θ)
            # Computing the derivative of the cartpole dynamics
            for i in range(self.state.ndx):
                data.Fx[i, i] = ADCG(CG(1.0))
            # Computing derivatives of the cost function
            w0_2, w1_2, w2_2, w3_2, w4_2, w5_2 = w[0] * w[0], w[1] * w[1], w[2] * w[2], w[3] * w[3], w[4] * w[4], w[5] * w[5]
            data.Lx[0] = w2_2 * y
            data.Lx[1] = w0_2 * sin_θ * cos_θ + w1_2 * (ADCG(CG(1.0)) - cos_θ) * sin_θ
            data.Lx[2] = w3_2 * ẏ
            data.Lx[3] = w4_2 * θ̇
            data.Lxx[0, 0] = w2_2
            data.Lxx[1, 1] = w0_2 * (cos_θ * cos_θ - sin_θ * sin_θ)
            data.Lxx[1, 1] += w1_2 * ((ADCG(CG(1.0)) - cos_θ) * cos_θ + sin_θ * sin_θ)
            data.Lxx[2, 2] = w3_2
            data.Lxx[3, 3] = w4_2

    def createData(self):
        """
        Creates the action data structure for the cart-pole model.

        Returns:
            ActionDataCartpole: Data structure to store intermediate computations.
        """
        return ActionDataCartpole(self)


class ActionDataCartpole(crocoddyl.cgfloat64.ActionDataAbstract):
    """
    Data structure for storing intermediate computations of the cart-pole dynamics.
    """
    def __init__(self, model):
        """
        Initializes the data structure with default values.

        Args:
            model (ActionModelCartpole): Cart-pole action model.
        """
        crocoddyl.cgfloat64.ActionDataAbstract.__init__(self, model)
        self.μ = ADCG(CG(0.0))
        self.ÿ̈ = ADCG(CG(0.0))
        self.θ̈ = ADCG(CG(0.0))
        self.ẏ_next = ADCG(CG(0.0))
        self.θ̇_next = ADCG(CG(0.0))
        self.y_next = ADCG(CG(0.0))
        self.θ_next = ADCG(CG(0.0))
        self.dμ_dθ = ADCG(CG(0.0))
        self.dÿ̈_dy = ADCG(CG(0.0))
        self.dÿ̈_dθ = ADCG(CG(0.0))
        self.dÿ̈_dθ = ADCG(CG(0.0))
        self.dÿ̈_dθ = ADCG(CG(0.0))
        self.dÿ̈_dẏ = ADCG(CG(0.0))
        self.dÿ̈_dθ̇ = ADCG(CG(0.0))
        self.dÿ̈_du = ADCG(CG(0.0))
        self.dθ̈_dy = ADCG(CG(0.0))
        self.dθ̈_dθ = ADCG(CG(0.0))
        self.dθ̈_dθ = ADCG(CG(0.0))
        self.dθ̈_dθ = ADCG(CG(0.0))
        self.dθ̈_dθ = ADCG(CG(0.0))
        self.dθ̈_dẏ = ADCG(CG(0.0))
        self.dθ̈_dθ̇ = ADCG(CG(0.0))
        self.dẏ_next_dy = ADCG(CG(0.0))
        self.dẏ_next_dθ = ADCG(CG(0.0))
        self.dẏ_next_dẏ = ADCG(CG(0.0))
        self.dẏ_next_dθ̇ = ADCG(CG(0.0))
        self.dẏ_next_du = ADCG(CG(0.0))
        self.dθ̇_next_dy = ADCG(CG(0.0))
        self.dθ̇_next_dθ = ADCG(CG(0.0))
        self.dθ̇_next_dẏ = ADCG(CG(0.0))
        self.dθ̇_next_dθ̇ = ADCG(CG(0.0))
        self.dθ̇_next_du = ADCG(CG(0.0))
        self.dy_next_dy = ADCG(CG(0.0))
        self.dy_next_dθ = ADCG(CG(0.0))
        self.dy_next_dẏ = ADCG(CG(0.0))
        self.dy_next_dθ̇ = ADCG(CG(0.0))
        self.dy_next_du = ADCG(CG(0.0))
        self.dθ_next_dy = ADCG(CG(0.0))
        self.dθ_next_dθ = ADCG(CG(0.0))
        self.dθ_next_dẏ = ADCG(CG(0.0))
        self.dθ_next_dθ̇ = ADCG(CG(0.0))
        self.dθ_next_du = ADCG(CG(0.0))

## Validating Analytical Derivatives with Automatic Differentiation (AutoDiff)

Once the action model is defined and its analytical derivatives for the dynamics and cost function are derived, it is crucial to verify their correctness. Since our action model supports code generation, we validate these derivatives using **automatic differentiation (AutoDiff)**. AutoDiff computes the Jacobian and Hessian by directly evaluating the expression and then generates an optimized C++ library, which is linked within `ActionModelCodeGen`.

Generating a C++ library through code generation can take a few seconds, depending on the complexity of the expressions. This process can be performed with or without AutoDiff. Below, we demonstrate how to generate action models both ways.

In [ ]:
# Code generate ActionModelCarpole without and with AutoDiff
model = crocoddyl.ActionModelCodeGen(ActionModelCartpole(), "cartpole")
data = model.createData()
model_ad = crocoddyl.ActionModelCodeGen(ActionModelCartpole(), "cartpole_ad", True)
data_ad = model_ad.createData()

# Generate a random state and control
x = model.state.rand()
u = np.random.rand(1)

# Compute analytical and NumDiff derivatives
model.calc(data, x, u)
model.calcDiff(data, x, u)
model_ad.calc(data_ad, x, u)
model_ad.calcDiff(data_ad, x, u)

# Compare derivatives
print("Fx difference:", np.linalg.norm(data.Fx - data_ad.Fx))
print("Fu difference:", np.linalg.norm(data.Fu - data_ad.Fu))
print("Lx difference:", np.linalg.norm(data.Lx - data_ad.Lx))
print("Lu difference:", np.linalg.norm(data.Lu - data_ad.Lu))
print("Lxx difference:", np.linalg.norm(data.Lxx - data_ad.Lxx))
print("Lxu difference:", np.linalg.norm(data.Lxu - data_ad.Lxu))

## Benchmarking Code-Generated Analytical vs. Automatic Differentiation

To assess the computational cost of AutoDiff, we compare its performance against code-generated analytical derivatives. Specifically, we run `calc` and `calcDiff` 1 millon times and profile the execution using Crocoddyl’s built-in profiling framework. The process is straightforward, as demonstrated below:

In [ ]:
import sys

# Number of trials
n_iter = 1000000

# Enable and reset the profiler
crocoddyl.enable_profiler()
crocoddyl.stop_watch_reset_all()

# Run calc and calcDiff without AutoDiff
for _ in range(n_iter):
    model.calc(data, x, u)
    model.calcDiff(data, x, u)

# Report the profiling results for CodeGen
crocoddyl.stop_watch_report(4)
sys.stdout.flush()

# Store the computation time associated to calc and calcDiff
t_calc = crocoddyl.stop_watch_get_total_time("ActionModelCodeGen::calc")
t_calcDiff = crocoddyl.stop_watch_get_total_time("ActionModelCodeGen::calcDiff")

# Enable and reset the profiler
crocoddyl.stop_watch_reset_all()
crocoddyl.enable_profiler()

# Run calc and calcDiff without AutoDiff
for _ in range(n_iter):
    model_ad.calc(data_ad, x, u)
    model_ad.calcDiff(data_ad, x, u)

# Store the computation time associated to calc and calcDiff
t_calc_ad = crocoddyl.stop_watch_get_total_time("ActionModelCodeGen::calc")
t_calcDiff_ad = crocoddyl.stop_watch_get_total_time("ActionModelCodeGen::calcDiff")

# Report the profiling results for AutoDiff
crocoddyl.stop_watch_report(4)
sys.stdout.flush()

ration_cg = t_calc_ad / t_calc
ration = t_calcDiff_ad / t_calcDiff
print()
print(f"Code generation with analytical derivatives is {ration_cg:.2f} times as fast as AutoDiff.")
print(f"Analytical derivatives are {ration:.2f} times as fast as AutoDiff.")

## Balancing our Cart-Pole System using Optimal Control

After verifying the analytical derivatives, we now solve the optimal control problem introduced in `cartpole_swingup.ipynb`, but this time using a code-generated library for computing cost, dynamics, and their derivatives.

It is worth noting that Crocoddyl’s Code Generation and AutoDiff framework supports both equality and inequality constraints. However, for simplicity, we ignore them in this example.

The optimal control problem is formulated as follows:

\begin{aligned}
& \underset{u(t)}{\text{minimize}} \quad J = \frac{1}{2} \sum_{i=0}^{N} w_i \, r_i^2(\mathbf{x}, \mathbf{u})  \\
& \text{subject to} \quad \mathbf{x}^+ = f(\mathbf{x}, \mathbf{u})
\end{aligned}

Where:
 - $J$ is the cost function, which sums over the trajectory nodes  i  the weighted square of the residual  r_i(\mathbf{x}, \mathbf{u}) , capturing the deviation from the desired state and control input.
 - $w_i$ is the weight associated with each node in the trajectory, typically representing how much importance we assign to a given time step in the optimization.
 - $\mathbf{x}^+$  represents the state at the next time step, while $\mathbf{x}$ is the current state.
 - $f(\mathbf{x}, \mathbf{u})$ represents the system’s dynamics, describing how the state evolves from one time step to the next under the control input $\mathbf{u}[i]$.

The implementation is shown below:

In [ ]:
from IPython.display import HTML
from cartpole_utils import animateCartpole
import sys

# Create a shooting problem with 50 running nodes (i.e., 50 * 5e-2 = 2.5 sec)
N = 50
x0 = np.array([0.0, 0.0, 1.0, 0.5])
problem = crocoddyl.ShootingProblem(x0, [model.copy() for _ in range(N)], model)

# Creating the FDDP solver and setting the logger callback
solver = crocoddyl.SolverFDDP(problem)
solver.setCallbacks([crocoddyl.CallbackVerbose()])

crocoddyl.enable_profiler()

# Solving this problem and display the optimal trajectory of our cart-pole system
solver.solve()

crocoddyl.stop_watch_report(4)
crocoddyl.stop_watch_reset_all()
sys.stdout.flush()

anim = animateCartpole(solver.xs)
# HTML(anim.to_jshtml())
HTML(anim.to_html5_video())

## Changing the Reference in Our Cart-Pole Action Model

In many applications, we need to dynamically update reference values. A common example is **model predictive control (MPC)**, where cost references or constraint activations change over time.

For a **code-generated action model**, we handle this by defining a *parameter environment*. This environment determines how reference vectors modify internal parameters within the action model. It is implemented through a function that updates the relevant parameters accordingly.

In our cart-pole example, we aim to adjust the cost weights dynamically, as shown below:

In [ ]:
# Define the parameters environemnt
def cost_weights_env(model, vector):
    model.costWeights[:] = vector

# Code generate ActionModelCarpole with parameters
model_env = crocoddyl.ActionModelCodeGen(ActionModelCartpole(), "cartpole", False, 6, cost_weights_env)
data_env = model_env.createData()

# Define the same parameters as before
model_env.update_p(data_env, np.array([1.0, 1.0, 0.1, 0.001, 0.001, 1.0]))

Introducing parameters in our code-generated action model typically has **minimal impact on computation time**, as demonstrated in the following benchmark:

In [ ]:
import sys

# Number of trials
n_iter = 1000000

# Enable and reset the profiler
crocoddyl.enable_profiler()
crocoddyl.stop_watch_reset_all()

# Run calc and calcDiff without parameters
for _ in range(n_iter):
    model.calc(data, x, u)
    model.calcDiff(data, x, u)

# Report the profiling results for parameters
crocoddyl.stop_watch_report(4)
sys.stdout.flush()

# Store the computation time associated to calc and calcDiff
t_calc = crocoddyl.stop_watch_get_total_time("ActionModelCodeGen::calc")
t_calcDiff = crocoddyl.stop_watch_get_total_time("ActionModelCodeGen::calcDiff")

# Enable and reset the profiler
crocoddyl.stop_watch_reset_all()
crocoddyl.enable_profiler()

# Run calc and calcDiff without AutoDiff
for _ in range(n_iter):
    model_env.calc(data_env, x, u)
    model_env.calcDiff(data_env, x, u)

# Store the computation time associated to calc and calcDiff
t_calc_ad = crocoddyl.stop_watch_get_total_time("ActionModelCodeGen::calc")
t_calcDiff_ad = crocoddyl.stop_watch_get_total_time("ActionModelCodeGen::calcDiff")

# Report the profiling results for AutoDiff
crocoddyl.stop_watch_report(4)
sys.stdout.flush()

ration_cg = t_calc / t_calc_ad * 100
ration = t_calcDiff / t_calcDiff_ad * 100
print()
print(f"Code generation with analytical derivatives is {ration_cg:.2f}% times faster than AutoDiff.")
print(f"Analytical derivatives are {ration:.2f}% times faster than AutoDiff.")

Finally, we update the cost weights for the terminal node. As expected, this modification alters the behavior compared to the previous solution.

In [ ]:
from IPython.display import HTML
from cartpole_utils import animateCartpole
import sys

# Create a shooting problem with 50 running nodes (i.e., 50 * 5e-2 = 2.5 sec)
N = 50
x0 = np.array([0.0, 0.0, 1.0, 0.5])
problem = crocoddyl.ShootingProblem(x0, [model_env.copy() for _ in range(N)], model_env)

# Change the cost weight associated to the terminal node
problem.terminalModel.update_p(problem.terminalData, np.array([100, 100., 1., 0.1, 0.01, 0.001]))

# Creating the FDDP solver and setting the logger callback
solver = crocoddyl.SolverFDDP(problem)
solver.setCallbacks([crocoddyl.CallbackVerbose()])

crocoddyl.enable_profiler()

# Solving this problem and display the optimal trajectory of our cart-pole system
solver.solve()

crocoddyl.stop_watch_report(4)
crocoddyl.stop_watch_reset_all()
sys.stdout.flush()

anim = animateCartpole(solver.xs)
# HTML(anim.to_jshtml())
HTML(anim.to_html5_video())

## What’s Next?

Now that we’ve explored how to solve an optimal control problem using FDDP and analyzed the benefits of code generation, the next step is to extend these concepts to more **general robotic systems**. In this notebook, we will cover:

1. **Action Model Code Generation**: Learn how to automatically code generate of complex action models needed to model robotics via the `ActionModelCodeGen` class.

2. **Derivatives and Sensitivity Analysis**: Validate code-generated derivatives by comparing them against code-generated automatic derivatives (e.g., using AutoDiff).

3. **Profiling and Performance Evaluation**: Compare the performance gains from AutoDiff derivatives vs. manually defined derivatives using code generation.

4. **Scaling to More Complex Systems**:	Understand how adapt reference in a code-generated action model.

By the end of the next notebook `scaling_to_robotics.ipynb`, you’ll have the skills to generate, validate, and optimize action models for your own robotics applications—unlocking faster and more accurate motion planning and control. 🚀